## Consolidate propagation results

In [2]:
import re
import string
import os, csv
import numpy as np
import pandas as pd
from _datetime import date
import json

In [3]:
# dfdirect = pd.read_csv('propagationDirectResults/2021-04-30_DirectHRtoFD.csv')
dfdirect = pd.read_csv('resultsPropagation/2021-07-01_DirectHRtoFD.csv')

dfdirect['snomedIdentifier'] = dfdirect['snomedIdentifier'].map(str)
dfdirect = dfdirect.rename(columns = {"snomedConcept_x":"snomedConcept",'snomedConcept_y':'snomedConceptHR'})
dfdirect.drop(['conditionName'], axis=1, inplace=True)
dfdirect['snomedIdentifierHR'] = dfdirect['snomedIdentifier']
# dfdirect.drop_duplicates(subset = ['snomedIdentifier'],keep = "first", inplace = True)
dfdirect['rule'] = '01_RULE'

dfdirect['propagation'] = len(dfdirect)
dfdirect['coverage'] = len(dfdirect)
dfdirect['query'] = 'DIRECT'
# dfdirect=dfdirect.reindex(columns= ['Rounded_score', 'Gender', 'Score','Name'])
dfdirect=dfdirect.reindex(columns= ['snomedConcept', 'snomedIdentifier', 'snomedConceptHR','snomedIdentifierHR','rule','propagation','coverage','query'])

print(len(dfdirect))
dfdirect.info()
dfdirect.head()

76
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76 entries, 0 to 75
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   snomedConcept       76 non-null     object
 1   snomedIdentifier    76 non-null     object
 2   snomedConceptHR     76 non-null     object
 3   snomedIdentifierHR  76 non-null     object
 4   rule                76 non-null     object
 5   propagation         76 non-null     int64 
 6   coverage            76 non-null     int64 
 7   query               76 non-null     object
dtypes: int64(2), object(6)
memory usage: 4.9+ KB


,snomedConcept,snomedIdentifier,snomedConceptHR,snomedIdentifierHR,rule,propagation,coverage,query
0,Acute bronchitis (disorder),10509002,Acute bronchitis (disorder),10509002,01_RULE,76,76,DIRECT
1,Radiation oncology AND/OR radiotherapy (proced...,108290001,Radiation oncology AND/OR radiotherapy (proced...,108290001,01_RULE,76,76,DIRECT
2,Concussion injury of brain (disorder),110030002,Concussion injury of brain (disorder),110030002,01_RULE,76,76,DIRECT
3,Cesarean section (procedure),11466000,Cesarean section (procedure),11466000,01_RULE,76,76,DIRECT
4,Disorder of kidney co-occurrent and due to dia...,127013003,Disorder of kidney co-occurrent and due to dia...,127013003,01_RULE,76,76,DIRECT


In [4]:
dfprop0 = pd.read_csv('resultsPropagation/2021-07-12_01NoEffectHRPropagation.csv')
dfprop0['snomedIdentifier'] = dfprop0['snomedIdentifier'].map(str)
dfprop0['snomedParentIdentifier'] = dfprop0['snomedParentIdentifier'].map(str)
dfprop0 = dfprop0.rename(columns = {'snomedParentConcept':'snomedConcept','snomedParentIdentifier':'snomedIdentifier',
                                    "snomedConcept":"snomedConceptHR",'snomedIdentifier':'snomedIdentifierHR'})
# dfprop4.drop(['snomedHRConcept','Index'], axis=1, inplace=True)
# # dfprop4.drop_duplicates(subset = ['snomedIdentifierHR'],keep = "first", inplace = True)
# dfprop3['rule'] = 'RULE0'
dfprop0=dfprop0.reindex(columns= ['snomedConcept', 'snomedIdentifier', 'snomedConceptHR','snomedIdentifierHR','rule','propagation','coverage','query'])
# dfprop0.loc[dfprop0['rule'] == '', 'predictedTag'] = 'NO EFFECT'
# dfprop0['predictedTag'] = 'NO EFFECT'
print(len(dfprop0))
dfprop0.info()
dfprop0.head()

152
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152 entries, 0 to 151
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   snomedConcept       152 non-null    object 
 1   snomedIdentifier    152 non-null    object 
 2   snomedConceptHR     152 non-null    object 
 3   snomedIdentifierHR  152 non-null    object 
 4   rule                152 non-null    object 
 5   propagation         152 non-null    float64
 6   coverage            152 non-null    float64
 7   query               152 non-null    object 
dtypes: float64(2), object(6)
memory usage: 9.6+ KB


,snomedConcept,snomedIdentifier,snomedConceptHR,snomedIdentifierHR,rule,propagation,coverage,query
0,Agreeing on diabetes care plan (procedure),703040004,Agreeing on diabetes care plan,703040004,02_RULE,1.0,1.0,IF DESCENDANT OF 703040004
1,Allergic disorder monitoring (regime/therapy),170836005,Allergic disorder follow-up assessment,170838006,02_RULE,3.0,3.0,IF DESCENDANT OF 170836005
2,Allergic disorder monitoring (regime/therapy),170836005,Allergic disorder initial assessment,170837001,02_RULE,3.0,3.0,IF DESCENDANT OF 170836005
3,Allergic disorder monitoring (regime/therapy),170836005,Allergic disorder monitoring,170836005,02_RULE,3.0,3.0,IF DESCENDANT OF 170836005
4,Allergy screening test (procedure),395142003,Allergy screening test,395142003,02_RULE,1.0,1.0,IF DESCENDANT OF 395142003


In [5]:
dfprop7 = pd.read_csv('resultsPropagation/2021-07-01_03DirectChildrenHRPropagation.csv')
dfprop7['snomedIdentifier'] = dfprop7['snomedIdentifier'].map(str)
dfprop7['snomedParentIdentifier'] = dfprop7['snomedParentIdentifier'].map(str)
dfprop7 = dfprop7.rename(columns = {'snomedParentConcept':'snomedConcept','snomedParentIdentifier':'snomedIdentifier',
                                    "snomedConcept":"snomedConceptHR",'snomedIdentifier':'snomedIdentifierHR'})
# # dfprop4.drop(['snomedHRConcept','Index'], axis=1, inplace=True)
# # # dfprop4.drop_duplicates(subset = ['snomedIdentifierHR'],keep = "first", inplace = True)
# # dfprop3['rule'] = 'RULE0'
# dfprop7=dfprop7.reindex(columns= ['snomedConcept', 'snomedIdentifier', 'snomedConceptHR','snomedIdentifierHR','rule','propagation','coverage','query'])
# # dfprop0.loc[dfprop0['rule'] == '', 'predictedTag'] = 'NO EFFECT'
# # dfprop0['predictedTag'] = 'NO EFFECT'
# print(len(dfprop7))
dfprop7.info()
dfprop7.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   query               11 non-null     object 
 1   rule                11 non-null     object 
 2   snomedConceptHR     11 non-null     object 
 3   snomedIdentifierHR  11 non-null     object 
 4   snomedConcept       11 non-null     object 
 5   snomedIdentifier    11 non-null     object 
 6   propagation         11 non-null     float64
 7   coverage            11 non-null     float64
dtypes: float64(2), object(6)
memory usage: 832.0+ bytes


,query,rule,snomedConceptHR,snomedIdentifierHR,snomedConcept,snomedIdentifier,propagation,coverage
0,DIRECT CHILDREN OF 32398004,03_RULE,Acute bronchitis (disorder),10509002,Bronchitis (disorder),32398004,19.0,1.0
1,DIRECT CHILDREN OF 423827005,03_RULE,Nasal sinus endoscopy (procedure),112790001,Endoscopy (procedure),423827005,54.0,1.0
2,DIRECT CHILDREN OF 250980009,03_RULE,Direct current cardioversion (procedure),180325003,Cardioversion (procedure),250980009,4.0,1.0
3,DIRECT CHILDREN OF 93880001,03_RULE,Small cell carcinoma of lung (disorder),254632001,Primary malignant neoplasm of lung (disorder),93880001,29.0,2.0
4,DIRECT CHILDREN OF 93880001,03_RULE,Non-small cell lung cancer (disorder),254637007,Primary malignant neoplasm of lung (disorder),93880001,29.0,2.0


In [6]:
dfprop1 = pd.read_csv('resultsPropagation/2021-07-12_04PropAttributesRules.csv')
dfprop1['descendantId'] = dfprop1['descendantId'].map(str)
dfprop1['snomedIdentifier'] = dfprop1['snomedIdentifier'].map(str)
# dfprop1['snomedParentIdentifier'] = dfprop1['snomedParentIdentifier'].map(int)
dfprop1 = dfprop1.rename(columns = {'descendantConcept':'snomedConceptHR', 'descendantId':'snomedIdentifierHR'})
dfprop1.drop(['snomedHRConcept'], axis=1, inplace=True)
# # # dfprop1['snomedIdentifierHR'] = dfprop1['snomedIdentifier'].map(int)
dfprop1['rule'] = '04_RULE'
dfprop1=dfprop1.reindex(columns= ['snomedConcept', 'snomedIdentifier', 'snomedConceptHR','snomedIdentifierHR','rule','propagation','coverage','query'])

print(len(dfprop1))

dfprop1.info()
dfprop1.head()

161
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 161 entries, 0 to 160
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   snomedConcept       161 non-null    object
 1   snomedIdentifier    161 non-null    object
 2   snomedConceptHR     161 non-null    object
 3   snomedIdentifierHR  161 non-null    object
 4   rule                161 non-null    object
 5   propagation         161 non-null    int64 
 6   coverage            161 non-null    int64 
 7   query               161 non-null    object
dtypes: int64(2), object(6)
memory usage: 10.2+ KB


,snomedConcept,snomedIdentifier,snomedConceptHR,snomedIdentifierHR,rule,propagation,coverage,query
0,Acute bronchitis (disorder),10509002,Acute bronchitis (disorder),10509002,04_RULE,29,1,(*): ([1..1]{363698007|Finding site (attribut...
1,Radiation oncology AND/OR radiotherapy (proced...,108290001,Combined chemotherapy and radiation therapy (p...,703423002,04_RULE,341,7,(*): ([1..1]{363703001|Has intent (attribute)...
2,Radiation oncology AND/OR radiotherapy (proced...,108290001,Brachytherapy of breast (procedure),447759004,04_RULE,341,7,(*): ([1..1]{363703001|Has intent (attribute)...
3,Radiation oncology AND/OR radiotherapy (proced...,108290001,Radiation therapy care (regime/therapy),385798007,04_RULE,341,7,(*): ([1..1]{363703001|Has intent (attribute)...
4,Radiation oncology AND/OR radiotherapy (proced...,108290001,Intracavitary brachytherapy (procedure),384692006,04_RULE,341,7,(*): ([1..1]{363703001|Has intent (attribute)...


In [7]:
dfprop2 = pd.read_csv('resultsPropagation/2021-07-12_05PropOneAttributeRules.csv')
dfprop2['descendantId'] = dfprop2['descendantId'].map(str)
dfprop2['snomedIdentifier'] = dfprop2['snomedIdentifier'].map(str)
dfprop2 = dfprop2.rename(columns = {"descendantConcept":"snomedConceptHR",'descendantId':'snomedIdentifierHR'})
dfprop2.drop(['snomedHRConcept'], axis=1, inplace=True)
# # # dfprop1['snomedIdentifierHR'] = dfprop1['snomedIdentifier'].map(int)
dfprop2['rule'] = '05_RULE'
dfprop2=dfprop2.reindex(columns= ['snomedConcept', 'snomedIdentifier', 'snomedConceptHR','snomedIdentifierHR','rule','propagation','coverage','query'])

print(len(dfprop2))
dfprop2.info()
dfprop2.head()

24
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   snomedConcept       24 non-null     object
 1   snomedIdentifier    24 non-null     object
 2   snomedConceptHR     24 non-null     object
 3   snomedIdentifierHR  24 non-null     object
 4   rule                24 non-null     object
 5   propagation         24 non-null     int64 
 6   coverage            24 non-null     int64 
 7   query               24 non-null     object
dtypes: int64(2), object(6)
memory usage: 1.6+ KB


,snomedConcept,snomedIdentifier,snomedConceptHR,snomedIdentifierHR,rule,propagation,coverage,query
0,Blindness AND/OR vision impairment level (diso...,105597003,Blindness due to type 2 diabetes mellitus (dis...,60951000119105,05_RULE,25,1,(105597003|Blindness AND/OR vision impairment ...
1,Dialysis procedure (procedure),108241001,Renal dialysis (procedure),265764009,05_RULE,8,1,(108241001|Dialysis procedure (procedure)| OR ...
2,Spirometry (procedure),127783003,Spirometry (procedure),127783003,05_RULE,6,1,(127783003|Spirometry (procedure)| OR <!127783...
3,Postoperative care (regime/therapy),133899007,Postoperative care (regime/therapy),133899007,05_RULE,1,1,(133899007|Postoperative care (regime/therapy)...
4,Burn care (regime/therapy),133901003,Burn care (regime/therapy),133901003,05_RULE,1,1,(133901003|Burn care (regime/therapy)| OR <!13...


In [8]:
dfprop3 = pd.read_csv('resultsPropagation/2021-07-12_06PropNoAttributeRules.csv')
dfprop3['descendantId'] = dfprop3['descendantId'].map(str)
dfprop3['snomedIdentifier'] = dfprop3['snomedIdentifier'].map(str)
dfprop3 = dfprop3.rename(columns = {"descendantConcept":"snomedConceptHR",'descendantId':'snomedIdentifierHR'})
dfprop3.drop(['snomedHRConcept'], axis=1, inplace=True)
# # # # dfprop1['snomedIdentifierHR'] = dfprop1['snomedIdentifier'].map(int)
dfprop3['rule'] = '06_RULE'
# dfprop3.drop_duplicates(subset = ['snomedIdentifierHR'],keep = "first", inplace = True)
dfprop3=dfprop3.reindex(columns= ['snomedConcept', 'snomedIdentifier', 'snomedConceptHR','snomedIdentifierHR','rule','propagation','coverage','query'])

print(len(dfprop3))
dfprop3.info()
dfprop3.head()

798
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 798 entries, 0 to 797
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   snomedConcept       798 non-null    object
 1   snomedIdentifier    798 non-null    object
 2   snomedConceptHR     798 non-null    object
 3   snomedIdentifierHR  798 non-null    object
 4   rule                798 non-null    object
 5   propagation         798 non-null    int64 
 6   coverage            798 non-null    int64 
 7   query               798 non-null    object
dtypes: int64(2), object(6)
memory usage: 50.0+ KB


,snomedConcept,snomedIdentifier,snomedConceptHR,snomedIdentifierHR,rule,propagation,coverage,query
0,Osteitis deformans (disorder),2089002,Pathological fracture due to osteoporosis (dis...,443165006,06_RULE,5315,10,((<<76069003|Disorder of bone (disorder)|) OR...
1,Osteitis deformans (disorder),2089002,Closed fracture of hip (disorder),359817006,06_RULE,5315,10,((<<76069003|Disorder of bone (disorder)|) OR...
2,Osteitis deformans (disorder),2089002,Fracture subluxation of wrist (disorder),263102004,06_RULE,5315,10,((<<76069003|Disorder of bone (disorder)|) OR...
3,Osteitis deformans (disorder),2089002,Fracture of forearm (disorder),65966004,06_RULE,5315,10,((<<76069003|Disorder of bone (disorder)|) OR...
4,Osteitis deformans (disorder),2089002,Osteoporosis (disorder),64859006,06_RULE,5315,10,((<<76069003|Disorder of bone (disorder)|) OR...


In [9]:
dfprop4 = pd.read_csv('resultsPropagation/2021-07-12_07PropSetParents.csv')
dfprop4['snomedIdentifier'] = dfprop4['snomedIdentifier'].map(str)
dfprop4['descendantId'] = dfprop4['descendantId'].map(str)
dfprop4 = dfprop4.rename(columns = {"descendantConcept":"snomedConceptHR",'descendantId':'snomedIdentifierHR'})
dfprop4.drop(['snomedHRConcept'], axis=1, inplace=True)
# # # dfprop4.drop_duplicates(subset = ['snomedIdentifierHR'],keep = "first", inplace = True)
dfprop4['rule'] = '07_RULE'
dfprop4=dfprop4.reindex(columns= ['snomedConcept', 'snomedIdentifier', 'snomedConceptHR','snomedIdentifierHR','rule','propagation','coverage','query'])

print(len(dfprop4))
dfprop4.info()
dfprop4.head()

51
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   snomedConcept       51 non-null     object
 1   snomedIdentifier    51 non-null     object
 2   snomedConceptHR     51 non-null     object
 3   snomedIdentifierHR  51 non-null     object
 4   rule                51 non-null     object
 5   propagation         51 non-null     int64 
 6   coverage            51 non-null     int64 
 7   query               51 non-null     object
dtypes: int64(2), object(6)
memory usage: 3.3+ KB


,snomedConcept,snomedIdentifier,snomedConceptHR,snomedIdentifierHR,rule,propagation,coverage,query
0,Acute bronchitis (disorder),10509002,Acute bronchitis (disorder),10509002,07_RULE,1,1,(<!111273006|Acute respiratory disease (disord...
1,Concussion injury of brain (disorder),110030002,Concussion injury of brain (disorder),110030002,07_RULE,1,1,(<!127295002|Traumatic brain injury (disorder)...
2,Cesarean section (procedure),11466000,Cesarean section (procedure),11466000,07_RULE,1,1,(<!66201006|Extraction of fetus (procedure)| A...
3,Disorder of kidney co-occurrent and due to dia...,127013003,Disorder of kidney due to diabetes mellitus (d...,127013003,07_RULE,1,1,(<!74627003|Complication due to diabetes melli...
4,Traumatic brain injury (disorder),127295002,Traumatic brain injury (disorder),127295002,07_RULE,1,1,(<!127294003|Traumatic AND/OR non-traumatic br...


In [10]:
# DEPRECATED CODES solution
# dfprop6 = pd.read_csv('intermediateData/2021-06-23_updatedSnmdIdentifier.csv')
dfprop6 = pd.read_csv('intermediateData/2021-07-03_HRNewUpdatedCodes.csv')
dfprop6['snomedChlIdentifierNew'] = dfprop6['snomedChlIdentifierNew'].map(str)
dfprop6['snomedIdentifierHR'] = dfprop6['snomedIdentifierHR'].map(str)
dfprop6['rule'] = '08_RULE'
dfprop6['propagation'] = 1
dfprop6['coverage'] = 1
dfprop6 = dfprop6.rename(columns = {'snomedChlConceptNew':'snomedConcept','snomedChlIdentifierNew':'snomedIdentifier',
                                   'newCode':'query'})
dfprop6.drop(['active'], axis=1, inplace=True)
dfprop6.info()
dfprop6.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   query               13 non-null     object
 1   snomedConcept       18 non-null     object
 2   snomedIdentifier    18 non-null     object
 3   snomedConceptHR     18 non-null     object
 4   snomedIdentifierHR  18 non-null     object
 5   rule                18 non-null     object
 6   propagation         18 non-null     int64 
 7   coverage            18 non-null     int64 
dtypes: int64(2), object(6)
memory usage: 1.2+ KB


,query,snomedConcept,snomedIdentifier,snomedConceptHR,snomedIdentifierHR,rule,propagation,coverage
0,{'POSSIBLY_EQUIVALENT_TO': ['782555009']},Allergy to cow's milk protein (finding),782555009,Allergy to dairy product,425525006,08_RULE,1,1
1,{'POSSIBLY_EQUIVALENT_TO': ['213020009']},Allergy to egg protein (finding),213020009,Allergy to eggs,91930004,08_RULE,1,1


### Final dataset

In [11]:
dfResult = pd.DataFrame()
dfResult = dfResult.append(dfdirect)
dfResult = dfResult.append(dfprop0)
dfResult = dfResult.append(dfprop1)
dfResult = dfResult.append(dfprop2)
dfResult = dfResult.append(dfprop3)
dfResult = dfResult.append(dfprop4)
dfResult = dfResult.append(dfprop7)
dfResult = dfResult.append(dfprop6)

dfResult.sort_values(by=['snomedIdentifierHR', 'rule','propagation'],inplace=True,ascending=[True,True, True])

dfResult.info()
dfResult.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1291 entries, 10 to 425
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   snomedConcept       1291 non-null   object 
 1   snomedIdentifier    1291 non-null   object 
 2   snomedConceptHR     1291 non-null   object 
 3   snomedIdentifierHR  1291 non-null   object 
 4   rule                1291 non-null   object 
 5   propagation         1291 non-null   float64
 6   coverage            1291 non-null   float64
 7   query               1286 non-null   object 
dtypes: float64(2), object(6)
memory usage: 90.8+ KB


,snomedConcept,snomedIdentifier,snomedConceptHR,snomedIdentifierHR,rule,propagation,coverage,query
10,Counseling (procedure),409063005,Counseling for termination of pregnancy (proce...,10383002,05_RULE,49.0,1.0,(409063005|Counseling (procedure)| OR <!409063...
704,Counseling (procedure),409063005,Counseling for termination of pregnancy (proce...,10383002,06_RULE,215.0,1.0,((<<128927009|Procedure by method (procedure)|...
705,Counseling (procedure),409063005,Counseling for termination of pregnancy (proce...,10383002,06_RULE,215.0,1.0,((<<128927009|Procedure by method (procedure)|...
33,"Hemogram, automated, with red blood cells, whi...",104091002,Hemoglobin / Hematocrit / Platelet count,104091002,02_RULE,1.0,1.0,IF DESCENDANT OF 104091002
695,Genetic test (procedure),405824009,Microbial culture of sputum (procedure),104173009,06_RULE,1857.0,7.0,((<<15220000|Laboratory test (procedure)|) OR...


In [12]:
dfResult.tail()

,snomedConcept,snomedIdentifier,snomedConceptHR,snomedIdentifierHR,rule,propagation,coverage,query
30,Nutritional disorder (disorder),2492009,Macular edema and retinopathy due to type 2 di...,97331000119101,06_RULE,78214.0,7.0,((<<64572001|Disease (disorder)|) OR <<249200...
372,Substance abuse (disorder),66214007,Macular edema and retinopathy due to type 2 di...,97331000119101,06_RULE,78214.0,7.0,((<<64572001|Disease (disorder)|) OR <<662140...
411,Mental disorder (disorder),74732009,Macular edema and retinopathy due to type 2 di...,97331000119101,06_RULE,78214.0,7.0,((<<64572001|Disease (disorder)|) OR <<747320...
418,Mental disorder (disorder),74732009,Macular edema and retinopathy due to type 2 di...,97331000119101,06_RULE,78214.0,7.0,((<<64572001|Disease (disorder)|) OR <<747320...
425,Poisoning (disorder),75478009,Macular edema and retinopathy due to type 2 di...,97331000119101,06_RULE,78214.0,7.0,((<<64572001|Disease (disorder)|) OR <<754780...


In [13]:
dfHRDifferent = dfResult.drop_duplicates(subset = ['snomedIdentifierHR'],keep = "first")
dfHRDifferent.info()
dfHRDifferent.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 380 entries, 10 to 610
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   snomedConcept       380 non-null    object 
 1   snomedIdentifier    380 non-null    object 
 2   snomedConceptHR     380 non-null    object 
 3   snomedIdentifierHR  380 non-null    object 
 4   rule                380 non-null    object 
 5   propagation         380 non-null    float64
 6   coverage            380 non-null    float64
 7   query               375 non-null    object 
dtypes: float64(2), object(6)
memory usage: 26.7+ KB


,snomedConcept,snomedIdentifier,snomedConceptHR,snomedIdentifierHR,rule,propagation,coverage,query
10,Counseling (procedure),409063005,Counseling for termination of pregnancy (proce...,10383002,05_RULE,49.0,1.0,(409063005|Counseling (procedure)| OR <!409063...
33,"Hemogram, automated, with red blood cells, whi...",104091002,Hemoglobin / Hematocrit / Platelet count,104091002,02_RULE,1.0,1.0,IF DESCENDANT OF 104091002
695,Genetic test (procedure),405824009,Microbial culture of sputum (procedure),104173009,06_RULE,1857.0,7.0,((<<15220000|Laboratory test (procedure)|) OR...
61,Measurement of Varicella-zoster virus antibody...,104326007,Measurement of Varicella-zoster virus antibody,104326007,02_RULE,3.0,1.0,IF DESCENDANT OF 104326007
35,Measurement of viral antibody (procedure),122435008,Hepatitis C antibody test,104375008,02_RULE,309.0,1.0,IF DESCENDANT OF 122435008


In [14]:
dfResult.to_csv('resultsPropagation/'+str(date.today())+'_propagationConsolidated.csv',index=False)